In [ ]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler 
# read in all our data
adult_income = pd.read_csv("/kaggle/input/adult-census-income/adult.csv")

adult_income

In [ ]:
#droping fnlwgt because it indicates number of people that a given observation represents
adult_income.drop('fnlwgt', axis=1, inplace=True)

# dropping education because it represents highest education which can be derived from education.num
adult_income.drop('education.num', axis=1, inplace=True)  
adult_income.drop('native.country', axis=1, inplace=True) 

# initial number of rows
adult_income.shape[0]

In [ ]:
adult_income.isin(['?']).sum()

In [ ]:
#drop rows with atleast one column value of ?
adult_income.replace('?', np.nan, inplace=True)
adult_income.dropna(inplace=True)
adult_income.drop_duplicates(inplace=True)

#new number of rows
adult_income.shape[0]

xx=adult_income

In [ ]:
# Encode categorical variables
categorical_cols = ['workclass', 'marital.status', 'occupation', 'relationship', 'race','sex']

adult_income = pd.get_dummies(adult_income, columns=categorical_cols)

le = LabelEncoder()
adult_income['education'] = le.fit_transform(adult_income['education'])

In [ ]:
adult_income['income'] = adult_income['income'].map({'<=50K':0, '>50K':1})
adult_income.income.head()

In [ ]:
# #normalize input data,ensure that each feature contributes equally to the analysis
# numerical_cols = ['age','capital.gain','capital.loss','hours.per.week']

# scaler = StandardScaler()

# adult_income[numerical_cols] = scaler.fit_transform(adult_income[numerical_cols])
# adult_income.head()

In [ ]:
#discretize age
age_bins = [0,15,18,25, 35, 45, 55, 65, np.inf]
age_labels = ['0-15','15-18','18-25' ,'25-35', '35-45', '45-55', '55-65', '65+']
xx['age_bins'] = pd.cut(xx['age'], bins=age_bins, labels=age_labels) 

# Drop the original columns
xx = xx.drop(['age'], axis=1)
 

#discretize capitalloss
loss_bins = [0,5, 1500, 1902, 2001, 2500, np.inf]
loss_labels = ['0-5l','5-1500l','1500-1902l' ,'1902-2001l', '2001-2500l', '2500+ll']
xx['capitalloss_bins'] = pd.cut(xx['capital.loss'], bins=loss_bins, labels=loss_labels, include_lowest=True) 

# Drop the original columns
xx = xx.drop(['capital.loss'], axis=1)


#discretize capital gain


gain_bins = [0, 5, 20000, np.inf]
gain_labels = ['0-5g', '5-20000g', '20000+g']
xx['capitalgain_bins'] = pd.cut(xx['capital.gain'], bins=gain_bins, labels=gain_labels, include_lowest=True) 

# Drop the original columns
xx = xx.drop(['capital.gain'], axis=1)

xx

 

#discretize capital gain


gain_bins = [0, 35, 40, 168]
gain_labels = ['part-time', 'full-time', 'over-time']
xx['hoursperweek_bins'] = pd.cut(xx['hours.per.week'], bins=gain_bins, labels=gain_labels, include_lowest=True) 

# Drop the original columns
xx = xx.drop(['hours.per.week'], axis=1)

xx

In [ ]:
# categorical_cols = ['hoursperweek_bins','capitalgain_bins','age_bins','capitalloss_bins']

# adult_income = pd.get_dummies(adult_income, columns=categorical_cols)
 

In [ ]:
# income_50k = adult_income[adult_income['income'] == '>50K']
# income_l50k = adult_income[adult_income['income'] == '<=50K']

# # Extract capital gains for each income category
# capital_gain_50k = income_50k['capital.gain']
# capital_gain_l50k = income_l50k['capital.gain']

# fig, axs = plt.subplots(2, 1)

# axs[0].hist(capital_gain_50k, edgecolor='black')
# axs[0].set_title('Distribution of Age for Income > 50K')

# axs[1].hist(capital_gain_l50k, edgecolor='black')
# axs[1].set_title('Distribution of Age for Income <= 50K')
# plt.tight_layout()

In [ ]:
adult_income.to_csv('preadult.csv', sep=',')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error,accuracy_score,precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [ ]:
opp=adult_income
cc=adult_income
dd=adult_income

In [ ]:
X = opp.drop('income', axis=1)
# X = X.drop('capital.gain', axis=1)
# X = X.drop('native.country', axis=1) 
y = opp.income

In [ ]:
# Create target object and call it y

# Create X
# features = ['age', 'workclass', 'fnlwgt', 'education', 'education.num',
#        'marital.status', 'occupation', 'relationship', 'race', 'sex',
#        'capital.gain', 'capital.loss', 'hours.per.week', 'native.country']
X = adult_income.drop('income', axis=1)
y = adult_income.income

In [ ]:
# adult_income[ 'capital.gain'].hist()

In [ ]:
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

In [ ]:
import matplotlib.pyplot as plt
rf_model = RandomForestClassifier(criterion='entropy', random_state=1,max_features='auto', max_depth=10 ,n_estimators=100                          )
# fit your model 'max_features': 'sqrt', 'n_estimators': 300
rf_model.fit(train_X, train_y)

# Calculate the mean absolute error of your Random Forest model on the validation data
rf_val_predictions = rf_model.predict(val_X)
rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)
print("Validation MAE for Random Forest Model: ", rf_val_mae)
a = accuracy_score(val_y, rf_val_predictions) 
p = precision_score(val_y, rf_val_predictions)
r = recall_score(val_y, rf_val_predictions)
print(f"a: {a:.3f}")
print(f"p: {p:.3f}")
print(f"r: {r:.3f}")

In [ ]:
# Get feature importances
importances = rf_model.feature_importances_

# Get feature names
feature_names = train_X.columns

# Sort feature importances in descending order
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
for f in range(train_X.shape[1]):
    print("%d. feature %d (%f): %s" % (f + 1, indices[f], importances[indices[f]], feature_names[indices[f]]))

# Plot the feature importances
plt.figure()
plt.title("Feature importances")
plt.bar(range(train_X.shape[1]), importances[indices], color="r", align="center")
plt.xticks(range(train_X.shape[1]), feature_names[indices], rotation=90)
plt.xlim([-1, train_X.shape[1]])
plt.show()

In [ ]:
train_predictions = rf_model.predict(train_X)
train_mae = mean_absolute_error(train_y, train_predictions)
print("Training MAE for Random Forest Model: ", train_mae)

In [ ]:
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
train_sizes, train_scores, test_scores = learning_curve(rf_model, train_X, train_y, cv=5, scoring='accuracy', n_jobs=-1)
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(train_sizes, train_mean, 'o-', color='r', label='Training score')
plt.plot(train_sizes, test_mean, 'o-', color='g', label='Validation score')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='r')
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1, color='g')
plt.xlabel('Training examples')
plt.ylabel('Score')
plt.title('Learning Curve')
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier(criterion='gini', random_state=21, max_depth=10,min_samples_leaf= 2, min_samples_split= 5)

dtree.fit(train_X, train_y)
tree_pred = dtree.predict(val_X)

print("Decision Tree accuracy: ", accuracy_score(val_y, tree_pred))
p = precision_score(val_y, tree_pred)
r = recall_score(val_y, tree_pred) 
print(f"p: {p:.3f}")
print(f"r: {r:.3f}")

In [ ]:
from sklearn.model_selection import cross_val_score 
# rf = DecisionTreeClassifier(random_state=21,max_depth=10)
rf = RandomForestClassifier(random_state=1,max_depth=10)

# # Define the parameter for DT
# params = {
#     'max_depth': [None, 5, 10],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'criterion': ['gini', 'entropy']
# }

# for RF
params = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion' :['gini', 'entropy']
}

from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(rf, param_grid=params, cv=5)
grid_search.fit(train_X, train_y)

best_params = grid_search.best_params_ 
best_score = grid_search.best_score_

print(best_params)
print(best_score)

In [ ]:
# importances = rf_model.feature_importances_
# print("Feature Importances: \n", dict(zip(features, importances)))

In [ ]:
a=xx

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth

from mlxtend.frequent_patterns import association_rules
dd=a
df_cat = dd[a.columns]



# Convert the dataframe to a list of lists
records = df_cat.values.tolist()
 
# Initialize a TransactionEncoder
te = TransactionEncoder() 

# Transform the dataset
te_data = te.fit_transform(records)
df_transformed = pd.DataFrame(te_data, columns=te.columns_)

# Apply FP-growth algorithm
frequent_itemsets = fpgrowth(df_transformed, min_support=0.5, use_colnames=True)

# Extract association rules
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.5, support_only=False, 
                         )

# Print frequent itemsets and rules
print(frequent_itemsets)
print(rules.head(10))

In [ ]:
opp_transformed

In [ ]:
#print(adult_income.dtypes)
cat_cols = adult_income.select_dtypes(['object', 'category']).columns.tolist()
print(cat_cols)

In [ ]:
### import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
dd=a
df_cat = dd[a.columns]



# Convert the dataframe to a list of lists
records = df_cat.values.tolist()
 
# Initialize a TransactionEncoder
te = TransactionEncoder() 

# Transform the dataset
te_data = te.fit_transform(records)
df_transformed = pd.DataFrame(te_data, columns=te.columns_) 
 

# Apply Apriori algorithm
frequent_itemsets = apriori(df_transformed, min_support=0.7, use_colnames=True)

# Extract association rules
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.9)
rules = rules.sort_values(by=['lift'], ascending=False)
# Print frequent itemsets and rules
print(frequent_itemsets)
print(rules)
print("hi")

# Print the names of the columns with their index position
for col in dd.columns:
    print(col)

In [ ]:
rules.to_csv('rules.csv', index=False)
frequent_itemsets.to_csv('freq.csv', index=False)

In [ ]:
ru=pd.read_csv('/kaggle/working/rules.csv')
fr=pd.read_csv('/kaggle/working/freq.csv')
